In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('Hotels_data_Changed.csv')

In [3]:
#get the top hotels
hotels=data.groupby('Hotel Name')
hotels=hotels.count().sort_values(['Checkin Date'],ascending=False).head(150)
hotelsNames=hotels.index

In [4]:
#get top checkin date
checkinDates=data.loc[data['Hotel Name'].isin(hotelsNames)].groupby(['Checkin Date']).size()
checkinDates=checkinDates.sort_values(ascending=False)
checkinDates=checkinDates.head(40)
checkinDates=checkinDates.index

In [5]:
#take only relevants cols and rows according top hotels and sort by price,drop duplicates
data=data.loc[data['Hotel Name'].isin(hotelsNames)].sort_values('Discount Price')
data=data[['Hotel Name','Checkin Date','Discount Code','Discount Price']]

In [6]:
data=data.drop_duplicates(['Checkin Date','Discount Code','Hotel Name']).sort_values(['Checkin Date','Hotel Name'])

In [7]:
#init clustering list with 150 rows and 161 cols while first col fit to top hotel and other cols init with -1
def initClusteringList(x,y):
        if x==0:
            return hotelsNames[y]
        else:
            return -1
initDate = [[initClusteringList(x,y) for x in range(161)] for y in range(150)]
initDate[0:2]

[['Newark Liberty International Airport Marriott',
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
 

In [8]:
#create list of top date and code(40dates*4codes)=>160 rows
datesAndCode=data.loc[data['Checkin Date'].isin(checkinDates)]
datesAndCode=datesAndCode.drop_duplicates(['Discount Code','Checkin Date'])

In [ ]:
#sort Checkin Date,Discount Code
datesAndCode=datesAndCode[['Checkin Date','Discount Code']].sort_values(['Checkin Date','Discount Code'])
len(datesAndCode.values)

160

In [ ]:
#update the table according merge with hotel name date and code from the main dataSet
def updateTable(x,y):
    query=datesAndCode.values[y]
    hotel=hotelsNames.values[x]
    line=data.loc[(data['Hotel Name']==hotel)&(data['Checkin Date']==query[0])&(data['Discount Code']==query[1])]
    if not line.empty:
        initDate[x][y+1]= line['Discount Price'].iloc[0]
for y in range(len(datesAndCode)):
    for x in range(hotelsNames.size):
        updateTable(x,y)

In [ ]:
len(initDate[0])

In [ ]:
df = pd.DataFrame(initDate)
df.to_csv("Top_Hotels_prices.csv",index=False,header=False)
#take only prices
normalizeData=df.loc[:,1:]

In [ ]:
import numpy as np
#update -1 vals to nan for normalization
def updateNotExistValToNan(x):
    if x==-1:
        return np.nan
    else:
        return x

normalizeDataTransform=normalizeData.applymap(updateNotExistValToNan)
normalizeDataTransform


In [ ]:
#normalize data per row according row max and min ignore nan and update nan to -1 as should be
def transformNormal(x):
    maxNum=np.nanmax(x)
    minNum=np.nanmin(x)
    diff=maxNum-minNum
    arr=[]
    for val in x:
        value=-1
        if(~np.isnan(val)):
            value=(((val-minNum)/diff)*100.00)
        arr.append(value)
    return arr
non=normalizeDataTransform.transpose().apply(lambda x:transformNormal(x))

In [ ]:
#transform back the data to 150*160 and add hotels names
allData=pd.DataFrame(non.transpose())
allData.fillna(-1,inplace=True)
hotels=pd.DataFrame(hotelsNames)
# hotels.combine_first(allData)
myNormalizedList=hotels.join(allData)
myNormalizedList.to_csv('Normalized_Data_panda.csv')

In [ ]:
myNormalizedList

In [ ]:
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
print (myNormalizedList.shape)

In [ ]:
X=pd.DataFrame(myNormalizedList.iloc[:,1:])
Z = linkage(X, 'average')

In [ ]:
#compares the actual pairwise distances samples to those implied by the hierarchical clustering
from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

c, coph_dists = cophenet(Z, pdist(X))
c

In [ ]:
# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('samples index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,
    show_contracted=True,
    leaf_font_size=8.,
)
plt.show()

In [ ]:
plt.title('Hierarchical Clustering Dendrogram(7 leaf)')
plt.xlabel('samples index')
plt.ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',
    p=7,
    leaf_rotation=90.,
    show_contracted=True,
    leaf_font_size=12.,
)
plt.show()

In [ ]:

# calculate full dendrogram
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram(7 leaf with labels)')
plt.xlabel('samples')
plt.ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',
    p=7,
    labels=hotelsNames,
    leaf_rotation=90.,
    show_contracted=True,
    leaf_font_size=12.,
)
plt.show()

In [ ]:
plt.title('Hierarchical Clustering Dendrogram(5 leaf)')
plt.xlabel('samples')
plt.ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',
    p=6,
    labels=hotelsNames,
    leaf_rotation=90.,
    show_contracted=True,
    leaf_font_size=12.)
plt.show()